# Actividad 1

In [14]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Cargar el conjunto de datos
archivo = 'Taller_2_Titulacion_DatosTaller.csv'
datos = pd.read_csv(archivo, encoding='latin-1', delimiter=';')

# Ver el número de valores faltantes por columna
valores_faltantes = datos.isnull().sum()
print("Valores faltantes por columna:")
print(valores_faltantes)

# Renombrar la columna 'Rotulo_Titulado' a 'Rotulo_Titulo' si es necesario
if 'Rotulo_Titulado' in datos.columns:
    datos = datos.rename(columns={'Rotulo_Titulado': 'Rotulo_Titulo'})

# Verificar si la columna 'Rotulo_Titulo' existe
if 'Rotulo_Titulo' not in datos.columns:
    raise ValueError("La columna 'Rotulo_Titulo' no se encuentra en el conjunto de datos.")

# Codificar la columna objetivo (si es necesario)
datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].apply(lambda x: 1 if x == 'SI' else 0)

# Verificar los valores únicos en la columna objetivo
print("Valores únicos en 'Rotulo_Titulo':", datos['Rotulo_Titulo'].unique())

# Identificar las columnas categóricas
columnas_categoricas = datos.select_dtypes(include=['object']).columns

# Codificar las variables categóricas
datos_codificados = pd.get_dummies(datos, columns=columnas_categoricas, drop_first=True)

# Normalizar las características numéricas
columnas_numericas = datos_codificados.select_dtypes(include=['float64', 'int64']).columns
escalador = StandardScaler()
datos_codificados[columnas_numericas] = escalador.fit_transform(datos_codificados[columnas_numericas])

# Guardar los datos preprocesados para la siguiente actividad
datos_codificados.to_csv('datos_preprocesados.csv', index=False)

print("Preprocesamiento completo y datos guardados en 'datos_preprocesados.csv'.")

Valores faltantes por columna:
Id                                                     0
MAT_1SEM_PROM                                          0
FIS_1SEM_PROM                                          0
ING_1SEM_PROM                                          0
ACTF_1SEM_A                                            0
OTRANS_1SEM_A                                          0
OTRANS_1SEM_R                                          0
OTRANS_1SEM_PROM                                       0
ESP_1SEM_A                                             0
ESP_1SEM_R                                             0
ESP_1SEM_PROM                                          0
INS_1SEM                                               0
PROM_1SEM                                              0
MAT_2SEM_PROM                                          0
FIS_2SEM_PROM                                          0
ING_2SEM_PROM                                          0
ACTF_2SEM_A                                            0


# Actividad 2

In [36]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold

# Cargar el conjunto de datos preprocesado
archivo = 'datos_preprocesados.csv'
datos = pd.read_csv(archivo)

# Verificar y corregir la columna objetivo si es necesario
if datos['Rotulo_Titulo'].dtype == object:
    datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].apply(lambda x: 1 if x == 'SI' else 0)

# Asegurarse de que 'Rotulo_Titulo' sea de tipo entero
datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].astype(int)

# Separar características y etiquetas
X = datos.drop('Rotulo_Titulo', axis=1)
y = datos['Rotulo_Titulo']

# Verificar nuevamente el tipo de y después de la codificación
print("Tipos únicos en 'Rotulo_Titulo':", y.unique())

# Definir el número de pliegues para la validación cruzada
num_pliegues = 5
validacion_cruzada = StratifiedKFold(n_splits=num_pliegues, shuffle=True, random_state=42)

# Iterar sobre los pliegues
for i, (train_index, val_index) in enumerate(validacion_cruzada.split(X, y)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
    
    # Aquí puedes entrenar tu modelo usando X_train, y_train
    # y evaluarlo usando X_val, y_val para cada pliegue
    
    print(f"Pliegue {i+1}: Tamaño de entrenamiento = {len(train_index)}, Tamaño de validación = {len(val_index)}")


Tipos únicos en 'Rotulo_Titulo': [ 0 -1]
Pliegue 1: Tamaño de entrenamiento = 2414, Tamaño de validación = 604
Pliegue 2: Tamaño de entrenamiento = 2414, Tamaño de validación = 604
Pliegue 3: Tamaño de entrenamiento = 2414, Tamaño de validación = 604
Pliegue 4: Tamaño de entrenamiento = 2415, Tamaño de validación = 603
Pliegue 5: Tamaño de entrenamiento = 2415, Tamaño de validación = 603


# Actividad 3

In [38]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Cargar los datos preprocesados
archivo = 'datos_preprocesados.csv'
datos = pd.read_csv(archivo)

# Verificar y corregir la columna objetivo si es necesario
if datos['Rotulo_Titulo'].dtype == object:
    datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].apply(lambda x: 1 if x == 'SI' else 0)

# Asegurarse de que 'Rotulo_Titulo' sea de tipo entero
datos['Rotulo_Titulo'] = datos['Rotulo_Titulo'].astype(int)

# Separar características y etiquetas
X = datos.drop('Rotulo_Titulo', axis=1)
y = datos['Rotulo_Titulo']

# Definir los modelos y los hiperparámetros a ajustar
modelos = {
    'Random Forest': {
        'modelo': RandomForestClassifier(),
        'parametros': {
            'n_estimators': [50, 100, 150],
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5],
            'max_features': ['sqrt', 'log2']
        }
    },
    'SVM': {
        'modelo': SVC(),
        'parametros': {
            'C': [1, 10, 100],
            'kernel': ['linear', 'rbf'],
            'gamma': ['scale', 'auto'],
            'degree': [2, 3]
        }
    }
}

# Ejecutar la búsqueda de hiperparámetros y seleccionar el mejor modelo para cada tipo
mejores_modelos = {}
for nombre, config in modelos.items():
    print(f"Optimizando {nombre}...")
    grid_search = GridSearchCV(config['modelo'], config['parametros'], cv=5, verbose=1, n_jobs=-1)
    grid_search.fit(X, y)
    mejores_modelos[nombre] = grid_search.best_estimator_
    print(f"Mejor configuración encontrada: {grid_search.best_params_}")
    print(f"Puntaje de validación cruzada: {grid_search.best_score_}\n")

# Seleccionar el mejor modelo basado en los resultados de validación cruzada
mejor_modelo = max(mejores_modelos, key=lambda x: grid_search.best_score_)
print(f"El mejor modelo encontrado es: {mejor_modelo}")
print(f"Con una precisión media de: {grid_search.best_score_}")


Optimizando Random Forest...
Fitting 5 folds for each of 36 candidates, totalling 180 fits


c:\Users\Vania\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
90 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
78 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Vania\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Vania\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\Vania\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.p

Mejor configuración encontrada: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 5, 'n_estimators': 150}
Puntaje de validación cruzada: 0.7524655969600123

Optimizando SVM...
Fitting 5 folds for each of 24 candidates, totalling 120 fits
